# Oura Downloader

Downloads data from Oura API and exports into CSV exports.

For more info on Oura API, see [Oura Documentation](https://cloud.ouraring.com/docs/)

----

### Dates Configuration

Configure start and end dates for collecting data

In [1]:
start_date="2020-01-01"
end_date="2020-12-31"

-----

## Dependencies and Libraries

In [2]:
BASE_URL = 'https://api.ouraring.com/v1/'

In [3]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime

------

## OPTION 1: Using Personal Access Token

This is the recommended appoach if you plan to use only on your own data. 

1. Login to https://cloud.ouraring.com/
2. Visit https://cloud.ouraring.com/personal-access-tokens and generate a personal access token. 
3. Copy you access token. 
4. Inside the directory, copy the file credentials-sample.json and create credentials.json
5. Open credentials.json in your editor, paste your Oura personal access token into "ACCESS_TOKEN" into credentials.json and save the file. NOTE: You can ignore the other options. 


## OPTION 2: Full API Integration
## Setup, Installation, Authentification Steps

#### Step 1:

* Go to the Developer Section of the Oura Cloud (https://cloud.ouraring.com/oauth/developer).
* Go to “My Applications” and create a new application by clicking on “New Application”.
* Fill out the form fields with your data.
* Set your callback URL as http://localhost:65010/oura_auth
* After saving the form, copy your "Client ID" and "Client Secret" from your Oura application, 
* Inside the directory, copy the file credentials-sample.json and create credentials.json
* Open credentials.json in your editor, paste your Oura "Client ID" and "Client Secret" into credentials.json and save the file. 
* Be sure to uncomment `%run ./oura_server.py`

#### Step 2: 

* NOTE: You will need to have Flask installed to run. You can install in commandline with `$ pip install flask`
* Run the following two cells below: CREDENTIALS STEP and TOKEN AUTHENTIFICATION STEP
* The first will load your current credentials
* The second will go through a token authentication process where you will need to visit: http://127.0.0.1:65010/ and click the link "Get access token!"
* Once completed and redirected back, copy your generated token. 
* Stop the current running process with control c or related command

#### Step 3: 

* Open credentials.json and paste in your token
* Comment out the line so it looks like this:
`# %run ./oura_server.py `
* Run all the cells to get your data.

## Authentification

In [4]:
# CREDENTIALS STEP
import json

with open("credentials.json", "r") as file:
    credentials = json.load(file)
    oura_cr = credentials['oura']
    CLIENT_SECRET = oura_cr['CLIENT_SECRET']
    CLIENT_ID = oura_cr['CLIENT_ID']
    ACCESS_TOKEN = oura_cr['ACCESS_TOKEN']

In [5]:
# TOKEN AUTHENTIFICATION STEP
# this script to start server 
# then visit provided url to complete process, generate token and copy
# Comment out once completed
# import flask
# %run ./oura_server.py

----

## User Info

In [6]:
url = BASE_URL+"userinfo?access_token="+ACCESS_TOKEN
r = requests.get(url)
user_info = r.json()

In [7]:
user_info

{'age': 37,
 'weight': 80.0,
 'height': 188,
 'gender': 'male',
 'email': 'markwkoester@gmail.com'}

-----

## Get Data Function

In [8]:
def get_oura_data(endpoint, start_date, end_date):
    endpoint=endpoint
    url = BASE_URL+endpoint+"?access_token="+ACCESS_TOKEN+"&start="+start_date+"&end="+end_date
    r = requests.get(url)
    data = r.json()    
    return data

----

## Sleep

In [9]:
sleep_data = get_oura_data(endpoint="sleep", start_date=start_date, end_date=end_date)

In [10]:
sleep = pd.DataFrame.from_dict(sleep_data['sleep'])

In [11]:
sleep['hours'] = round((sleep.total/60/60),2)

In [12]:
# Sleep Summary
print('{:,} days of tracked sleep'.format(len(sleep)))
print('{:,} median hours of sleep per night'.format(round((sleep.hours).median(),2)))

print('{:,} average hours of sleep per night'.format(round((sleep.hours).mean(),2)))
print('{:,} average sleep score'.format(round((sleep.score_total).mean(),1)))
print('{:,} average sleep efficiency'.format(round((sleep.efficiency).mean(),1)))
print('{:,} average hr'.format(round((sleep.hr_average).mean(),1)))
print('{:,} average breaths per minute'.format(round((sleep.breath_average).mean(),1)))


18 days of tracked sleep
7.38 median hours of sleep per night
7.25 average hours of sleep per night
79.1 average sleep score
88.3 average sleep efficiency
46.9 average hr
13.9 average breaths per minute


In [13]:
# sleep.tail()

In [14]:
# save to csv
sleep.to_csv('data/daily_sleep.csv', index=None, encoding='utf-8')

----

## Activity

In [15]:
activity_data = get_oura_data(endpoint="activity", start_date=start_date, end_date=end_date)

In [16]:
activity = pd.DataFrame.from_dict(activity_data['activity'])

In [17]:
# activity.tail()

In [18]:
# Activity Summary
print('{:,} days of tracked activity'.format(len(activity)))
print('{:,} average activity score per day'.format(round((activity.score).mean(),1)))
print('{:,} average daily movement (or equivalent to steps) per day'.format(round((activity.daily_movement).mean(),2)))
print('{:,} average of steps per day'.format(round((activity.steps).mean(),2)))
print('{:,} average energy consumption during the day'.format(round((activity.cal_total).mean(),2)))
print('=================')
print('{:,} average active hours (high)'.format(round((activity.high/60).mean(),1)))
print('{:,} average active hours(medium)'.format(round((activity.medium/60).mean(),1)))
print('{:,} average active hours (low)'.format(round((activity.low/60).mean(),1)))
print('{:,} average rest hours'.format(round((activity.rest/60).mean(),1)))
print('{:,} average inactive hours'.format(round((activity.inactive/60).mean(),1)))
print('{:,} average non_wear hours'.format(round((activity.non_wear/60).mean(),1)))

19 days of tracked activity
79.0 average activity score per day
7,590.16 average daily movement (or equivalent to steps) per day
9,115.42 average of steps per day
2,617.37 average energy consumption during the day
0.1 average active hours (high)
0.6 average active hours(medium)
4.5 average active hours (low)
7.6 average rest hours
5.8 average inactive hours
4.5 average non_wear hours


In [19]:
# save to csv
activity.to_csv('data/daily_activity.csv', index=None, encoding='utf-8')

----

## Readiness

In [20]:
readiness_data = get_oura_data(endpoint="readiness", start_date=start_date, end_date=end_date)

In [21]:
readiness = pd.DataFrame.from_dict(readiness_data['readiness'])

In [22]:
# readiness.tail()

In [23]:
# save to csv
readiness.to_csv('data/daily_readiness.csv', index=None, encoding='utf-8')